ดูว่าจังหวะตัดตอนพูดต้องตัดตอนกี่วิ split chunk

overlap text ที่ merge กันจะเคลียยังไง

โมเดลรับไฟล์เสียงสั้นสุดเท่าไร โมเดลถูกจูนมาแล้วให้รับไฟล์เสียงสั้นได้

อัดเสียงยาวๆแล้วซอยไฟล์ให้มัน overlap กับแล้วเทส

ออกแบบจัดเก็บไฟล์เสียงเป็น temp file

lib สร้าง temp file

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from package.typhoon.inference import TyphoonASR

d:\Git\thai_voice_transcription\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[NeMo W 2025-11-19 00:00:04 nemo_logging:393] d:\Git\thai_voice_transcription\.venv\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    


In [3]:
model = TyphoonASR()

🌪️ Loading Typhoon ASR model on CPU...
[NeMo I 2025-11-19 00:00:08 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 2048 tokens


[NeMo W 2025-11-19 00:00:09 nemo_logging:393] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/workspace/warit/nemo-asr/stt_th_conformer_transducer_large/prepare_data/typhoon_cleanser/20250814/Split_gg/train_data_typhoon_asr_realtime.jsonl
    sample_rate: 16000
    batch_size: 8
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 30.0
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: fully_randomized
    bucketing_batch_size: null
    
[NeMo W 2025-11-19 00:00:09 nemo_logging:393] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation d

[NeMo I 2025-11-19 00:00:09 nemo_logging:381] PADDING: 0


[NeMo W 2025-11-19 00:00:09 nemo_logging:393] d:\Git\thai_voice_transcription\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
      warnings.warn(
    


[NeMo I 2025-11-19 00:00:10 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-11-19 00:00:10 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}


[NeMo W 2025-11-19 00:00:10 nemo_logging:393] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: CUDA is not available


[NeMo I 2025-11-19 00:00:10 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}


[NeMo W 2025-11-19 00:00:10 nemo_logging:393] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: CUDA is not available


[NeMo I 2025-11-19 00:00:10 nemo_logging:381] Model EncDecRNNTBPEModel was successfully restored from C:\Users\terjr\.cache\huggingface\hub\models--scb10x--typhoon-asr-realtime\snapshots\a14b79d50c788dbdfe559c8a28a9b90153cf3865\typhoon-asr-realtime.nemo.


In [4]:
import glob

def merge_overlapped_text(texts):
    if not texts:
        return ""
    
    merged = texts[0]
    
    for i in range(1, len(texts)):
        current = texts[i]
        if not current:
            continue
            
        # หาส่วนที่ซ้ำกันระหว่างท้าย merged กับต้น current
        best_overlap = 0
        for j in range(1, min(len(merged), len(current)) + 1):
            if merged[-j:] == current[:j]:
                best_overlap = j
        
        # รวมโดยตัดส่วนที่ซ้ำออก
        merged += current[best_overlap:]
    
    return merged

# ถอดเสียงทีละ chunk
chunk_files = sorted(glob.glob("audio/chunks/*.wav"))
texts = []

for chunk_file in chunk_files:
    processed = model.preprocess(chunk_file)
    result = model.transcribe(processed)
    text = result['text'][0] if result['text'] else ''
    texts.append(text)
    print(text)

# รวมข้อความ
final_text = merge_overlapped_text(texts)
print(f"\noutput: {final_text}")


   Resampled: 192000 Hz → 16000 Hz
✅ Saved: processed_audio.wav (2.0s)


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s][NeMo W 2025-11-19 00:00:14 nemo_logging:393] d:\Git\thai_voice_transcription\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
      warnings.warn(warn_msg)
    
[NeMo W 2025-11-19 00:00:15 nemo_logging:393] Function ``_transcribe_output_processing`` is deprecated. The return type of args will be updated in the upcoming release to ensure a consistent output format across all decoder types, such that a "Hypothesis" object is always returned.
Transcribing: 100%|██████████| 1/1 [00:00<00:00,  8.81it/s]


ให้ตกน้ําตกล
   Resampled: 192000 Hz → 16000 Hz
✅ Saved: processed_audio.wav (2.0s)


Transcribing: 100%|██████████| 1/1 [00:00<00:00,  9.52it/s]


ตกรถตกเรือตกท่า
   Resampled: 192000 Hz → 16000 Hz
✅ Saved: processed_audio.wav (2.0s)


...

ไม่พบรักทุก
   Resampled: 192000 Hz → 16000 Hz
✅ Saved: processed_audio.wav (2.0s)


Transcribing: 100%|██████████| 1/1 [00:00<00:00, 10.86it/s]


ทุกชาติทุกชาติ
   Resampled: 192000 Hz → 16000 Hz
✅ Saved: processed_audio.wav (1.4s)


Transcribing: 100%|██████████| 1/1 [00:00<00:00, 12.92it/s]

ทุกชาติไป

output: ให้ตกน้ําตกลตกรถตกเรือตกท่าถ้าน้ําตกเครื่องบินตกโต๊ะตกเตียงตกลู้ตกสอบตกให้ค้างคบคกกัดให้งูรัฐให้โดนพัดลมบาทมือบาทเดียวเดี๋ยวและบาดแขนบาดขาคาสิลขึ้นหน้าฝ้าขึ้นหลังลามขึ้นคอยังไม่พอขอให้อกหักไม่พบรักทุกชาติทุกชาติไป
